In [ ]:
import pandas as pd
path = "./vast/raw_train_all_onecol.csv"
data = pd.read_csv(path)
data

In [ ]:


from transformers import RobertaModel, RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
text = '[CLS]'
sample_texts = ["I love this movie!", "I hate this movie."]
sample_stance = ["movie","movie"]
labels = [1, 0]  # 1: positive, 0: negative

# Encoding the texts
encodings = tokenizer(sample_texts, truncation=False, padding=False, return_tensors="pt")
print(encodings["input_ids"])
encodings_stance = tokenizer(sample_stance,truncation=False, padding=False,return_tensors="pt")
print(encodings["input_ids"])


In [4]:
import openai
def generate_answer(prompt):
    client = openai.OpenAI(
        api_key=""
        )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="o1-preview-2024-09-12",
        stream=False,
    )
    chat_completion = chat_completion.choices[0].message.content
    return chat_completion

generate_answer("hello")

'Hello! How can I assist you today?'

In [ ]:
from scripts.roberta_mlp import RoBERTa_MLP
import torch

# 初始化模型架构
model = RoBERTa_MLP(num_labels=3)

# 加载保存的权重
model_path = "./scripts/results/checkpoint-106/"
state_dict = torch.load(f"{model_path}/model.safetensors")
model.load_state_dict(state_dict)

In [ ]:
from safetensors.torch import load_file
from scripts.roberta_mlp import RoBERTa_MLP

# 初始化模型
model = RoBERTa_MLP(num_labels=3)

# 加载 .safetensors 权重
model_path = "./scripts/results/checkpoint-106/model.safetensors"
state_dict = load_file(model_path)

# 将加载的权重加载到模型中
model.load_state_dict(state_dict)
model.forward()


In [ ]:
import json
import pandas as pd

# path = "./data/wrong_test_data_424.json"
# with open(path,"r") as f:
#     data = json.load(f)
# print(len(data))
# data
path = """vast/raw_{}_all_onecol.csv"""
data = pd.read_csv(path.format("train"))
# print(len(data))
# str(data[:1])
data

In [ ]:
! pip install openai toml

In [ ]:
import openai
import toml
import pandas as pd
import json
from typing import List
import copy
args = toml.load("key.toml")
path = "./data/wrong_test_data_424.json"
with open(path,"r") as f:
    data = json.load(f)

openai.api_key = args["gpt-cot"]["openai_api_key"]
batch = 30
def generate_answer(prompt):
    client = openai.OpenAI(
        api_key=openai.api_key
        )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
        stream=False,
        temperature=0,
    )
    chat_completion = chat_completion.choices[0].message.content
    return chat_completion
def prepare_data(data:List[dict],num,batch):
    d = copy.deepcopy(data[num*batch:(num+1)*batch])
    for i in d:
        try:
            del i["index"]
        except Exception as e:
            print("error")
    return str(d)
prompt = """
You are an expert in multi-level weakness detection and augmentation. 
I will provide you with {batch} pieces of data where other models have given incorrect answers, 
related to stance detection. The label mapping is as follows: {{ "FAVOR": 0, "NONE": 1, "AGAINST": 2 }}
Please identify the similarities in the data.Data as follow.
{data}
please identify the similarities in the data.
"""
# print(prepare_data(data,1,batch))
# answer = generate_answer(prompt.format(batch=batch,data=prepare_data(data,batch)))


In [ ]:
# print(answer)
answer = []
for i in range(len(data)//batch):
    response = generate_answer(prompt.format(batch=batch,data=prepare_data(data,i,batch)))
    answer.append(response)
    print("-"*20)
    print(f"index : from {i*batch} to {(i+1)*batch}")
    print(response)
    print()
    

In [ ]:
import json
with open("results/get_response_mistake.json","w") as f:
    json.dump(answer,f,indent=4)

In [ ]:
prompt = """
You will be provided with five responses regarding stance detection. 
Please identify the common points in the following responses.
{responses}
"""
answer_batch = []
for i in range(len(answer)//5+1):
    x = generate_answer(prompt.format(responses = str(answer[i*5:(i+1)*5])))
    answer_batch.append(x)
    print("-"*20)
    print(f"index : from {i*5} to {(i+1)*5}")
    print(x)
    print()


In [ ]:
prompt = """
You will be provided with five responses regarding stance detection. 
Please identify the common points in the following responses.
{responses}
"""
print(generate_answer(prompt.format(responses=answer_batch)))

In [ ]:
prompt = """
You are an expert in multi-level weakness detection and augmentation. 
I will provide you with {batch} pieces of data where other models have given incorrect answers 
related to stance detection. The label mapping is as follows: {{"FAVOR": 0, "NONE": 1, "AGAINST": 2}}.
Please identify the similarities in the data and explain why these similarities arise. The data is as follows:
{data}
Please identify the similarities in the data.
"""
# print(prepare_data(data,0,batch))
answer = generate_answer(prompt.format(batch=batch,data=prepare_data(data,0,batch)))
print(answer)

In [ ]:
prompt = """
You are an expert instructor specializing in educational diagnostics and performance analysis. Your task is to assess student performance on {task_name}.

Task Context:
{task_description}

I will present you with {batch} instances where the student provided incorrect responses. The classification labels are defined as follows: {{"FAVOR": 0, "NONE": 1, "AGAINST": 2}}.

Please conduct a thorough analysis of the student's systematic errors and learning gaps. For each identified weakness, provide:

1. A detailed description of the error pattern
2. Potential underlying causes
3. Specific examples illustrating the weakness
4. Suggestions for generating targeted practice materials
5. Based on each weaknesses you have identified, please generate 100 similar sets of questions.

Format your analysis using XML tags, with each distinct weakness enclosed in <Weakness i></Weakness i> tags, 
where i represents the sequential number of the weakness. And use <Error Pattern> <PotantialCauses>, <Example>, <Sugestion>, <TestSet> to detail each weakness. 
The <PotantialCauses> should directly indicate what kinds of text or semantic information could cause such failure.
The <TestSet> should include {test_num} questions, each enclosed in <test i>, where 'i' is the sequential number of the test set. 
The <text>, <target>, and <ground_truth> elements should be within each <test i>.

Your analysis should be comprehensive enough to guide the creation of tailored educational materials that directly address these learning gaps.

The data are as follows:
{data}
"""
test_num = 100

task_name = "stance dection"

task_description = "Sentiment classification of comments on tweets."

batch = 30

# print(prompt.format(task_name = task_name, task_description = task_description, batch = batch, data=prepare_data(data,0,batch)))
answer = generate_answer(prompt.format(test_num=test_num,task_name = task_name, task_description = task_description, batch=batch, data=prepare_data(data,0,batch)))
print(answer)




In [ ]:
import xml.etree.ElementTree as ET
import json
# 解析XML数据
root = ET.fromstring(answer.strip("```xml").strip("```"))

# 定义一个函数将XML转换为JSON格式
def xml_to_json(element):
    json_data = {}
    for child in element:
        # 获取子元素的标签名作为key，子元素内容作为value
        json_data[child.tag] = child.text.strip() if len(child) == 0 else xml_to_json(child)
    return json_data

# 提取根元素
json_result = xml_to_json(root)

# 打印JSON格式的结果
print(json.dumps(json_result, indent=4))

In [ ]:
weakness = """
<Weakness1>
    <ErrorPattern>The student frequently misclassifies comments that express a nuanced or complex opinion as being against the topic, rather than recognizing them as neutral or favoring the topic.</ErrorPattern>
    <PotentialCauses>The student may struggle with identifying subtle sentiment cues in the text, particularly when the language used is critical but not outright negative. They might also have difficulty distinguishing between sarcasm or critique and genuine opposition.</PotentialCauses>
    <Example>The comment about the need for intercity passenger rail discusses costs and safety but is misclassified as "NONE" instead of "FAVOR".</Example>
    <Suggestion>Practice materials should include examples of nuanced opinions, asking students to identify whether the sentiment is favoring, neutral, or against the topic. Include discussions on sarcasm and critique in sentiment analysis.</Suggestion>
</Weakness1>
"""

prompt = """
You are an expert at generating data based on weaknesses and causes. 
Given the data with tags like ErrorPattern, PotentialCauses, Example, and Suggestion, 
generate {num_examples} examples using the following format:

<Example i>
    <text>Text content</text>
    <target>Entity in the text</target>
    <truth_label>Truth label (correct classification)</truth_label>
</Example i>

Provide the generated examples based on the provided {weakness} data.
"""
num_examples = 100
answer_weakness = generate_answer(prompt.format(num_examples=num_examples,weakness=weakness))
print(answer_weakness)


# 单条数据生成testset

In [ ]:
prompt = """
You are an expert instructor specializing in educational diagnostics and performance analysis. Your task is to assess student performance on {task_name}.

Task Context:
{task_description}

I will present you with an instance where the student provided incorrect responses. The classification labels are defined as follows: {{"FAVOR": 0, "NONE": 1, "AGAINST": 2}}.

Please conduct a thorough analysis of the student's systematic errors and learning gaps. For each identified weakness, provide:

1. A detailed description of the error pattern
2. Potential underlying causes
3. Based on each weaknesses you have identified, please generate 100 similar sets of questions.

Format your analysis using XML tags, with each distinct weakness enclosed in <Weakness i></Weakness i> tags, 
where i represents the sequential number of the weakness. And use <Error Pattern> <PotantialCauses>, <TestSet> to detail each weakness. 
The <PotantialCauses> should directly indicate what kinds of text or semantic information could cause such failure.
The <TestSet> should include {test_num} questions, each enclosed in <test i>, where 'i' is the sequential number of the test set. 
The <text>, <target>(Entity in the text), and <ground_truth>(Truth label) elements should be within each <test i>.
Ensure that the length of the text varies (the length exceeds 100)and that the diversity of the text is also guaranteed (you can choose materials from different fields),
and the text should be in line with <Weakness i>.

Your analysis should be comprehensive enough to guide the creation of tailored educational materials that directly address these learning gaps.

The data are as follows:
{data}
"""
batch = 1
test_num = 100

task_name = "stance detection"

task_description = "Stance detection aims to identify the authors' attitudes or positions [FAVOR, NONE, AGAINST] towards a specific target such as an entity, a topic."



print(prompt.format(test_num=test_num,task_name = task_name, task_description = task_description, data=prepare_data(data,0,batch)))
answer = generate_answer(prompt.format(test_num=test_num,task_name = task_name, task_description = task_description, data=prepare_data(data,0,batch)))
print(answer)


In [ ]:

import re
import xml.etree.ElementTree as ET
import json
import openai
import toml
import pandas as pd
import json
from typing import List
import copy
args = toml.load("key.toml")
path = "./data/wrong_test_data_424.json"

with open(path,"r") as f:
    data = json.load(f)
print("len(data): ", len(data))

openai.api_key = args["gpt-cot"]["openai_api_key"]

def generate_answer(prompt):
    client = openai.OpenAI(
        api_key=openai.api_key
        )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
        stream=False,
        temperature=0,
    )
    chat_completion = chat_completion.choices[0].message.content
    return chat_completion
def prepare_data(data:List[dict],num):
    d = copy.deepcopy(data[num])
    # for i in d:
    #     try:
    #         del i["index"]
    #     except Exception as e:
    #         print("error",e)
    del d["index"]
    return str(d)
def get_answer(data,num_data:int):    
    prompt = """
    You are an expert instructor specializing in educational diagnostics and teaching. Your task is to assess student performance on {task_name} 
    and provide the training instances to teach the student to become more expert.

    Task Context:
    {task_description}

    I will present you with an instance where the student provided incorrect responses. The classification labels are defined as follows: {{"FAVOR": 0, "NONE": 1, "AGAINST": 2}}.

    Please consider the potential weakness of the student considering the incorrect response and provide a training instances to teach it to be a more expert problem solver. 

    Format your training instances using XML tags.
    The <TrainingSet> should include {training_num} questions, each enclosed in <train i>, where 'i' is the sequential number of the training set. 
    The <text>, <target>(an entity in the text), and <ground_truth>(Truth label) elements should be within each <train i>.

    Ensure that the length of the text varies (the length exceeds 100) and that the diversity of the text is also guaranteed 
    (you can choose materials from different fields or different tones like diverse users on the website).

    The data are as follows:
    {data}
    """

    batch = 1
    training_num = 10

    task_name = "stance detection"

    task_description = "Stance detection aims to identify the authors' attitudes or positions [FAVOR, NONE, AGAINST] towards a specific target such as an entity, a topic."

    # print(prompt.format(training_num=training_num, task_name = task_name, task_description = task_description, data=prepare_data(data,num_data)))
    answer = generate_answer(prompt.format(training_num=training_num, task_name = task_name, task_description = task_description, data=prepare_data(data,num_data)))
    # print(answer)
    return answer

def parse(answer:str):
    pattern = r"```xml(.*?)```"
    match = re.search(pattern, answer, re.DOTALL)
    if match:
        extracted_content = match.group(1).strip()
        print("Extracted Content:")
        print(extracted_content[:20],".....")
    else:
        print("No content found between ```xml and ```")
    try:
        root = ET.fromstring(extracted_content)
    except Exception as e:
        print("error: ", answer[:40])
        return [answer]
    js = []
    for i in root:
        k = {}
        for j in i:
            k[j.tag] = j.text
        js.append(k)
    print("parse successfully: len(parse_json) = ",len(js))
    return js
result = []
complete_num = 0
for i in range(len(data)):
    print("-"*30)
    print(f"index: {i}, complete_num: {complete_num}")
    d = get_answer(data,i)
    if len(d) > 3:
        complete_num += 1
    result.append(parse(d))

with open("./data/all_error_instances_weakness_generate_train_dataset.json","w") as f:
    json.dump(result,f,indent=4)

In [ ]:
a = [1,2]
b = [1]
c = []

In [ ]:
import re
import xml.etree.ElementTree as ET
import json
pattern = r"```xml(.*?)```"
match = re.search(pattern, answer, re.DOTALL)
if match:
    extracted_content = match.group(1).strip()
    print("Extracted Content:")
    print(extracted_content[:14],".....")
else:
    print("No content found between ```xml and ```")
root = ET.fromstring(extracted_content)
js = []
for i in root:
    # print("<",i.tag,">")
    k = {}
    for j in i:
        # print(" <",j.tag,">")
        # print(j.text)
        k[j.tag] = j.text
    js.append(k)
print(js)
# with open("./data/sample_instance_weakness_generate_instances.json","w") as f:
#     json.dump(js,f,indent=4)

In [ ]:
import re
import xml.etree.ElementTree as ET
import json
import openai
import toml
import pandas as pd
import json
from typing import List
import copy
args = toml.load("key.toml")

with open("./data/all_error_instances_weakness_generate_train_dataset.json","r") as f:
    response_data = json.load(f)

path = "./data/wrong_test_data_424.json"
with open(path,"r") as f:
    raw_data = json.load(f)

openai.api_key = args["gpt-cot"]["openai_api_key"]

def generate_answer(prompt):
    client = openai.OpenAI(
        api_key=openai.api_key
        )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4o-mini",
        stream=False,
        temperature=0,
    )
    chat_completion = chat_completion.choices[0].message.content
    return chat_completion
def prepare_data(data:List[dict],num):
    d = copy.deepcopy(data[num])
    # for i in d:
    #     try:
    #         del i["index"]
    #     except Exception as e:
    #         print("error",e)
    del d["index"]
    return str(d)
def get_answer(data,num_data:int):    
    prompt = """
    You are an expert instructor specializing in educational diagnostics and teaching. Your task is to assess student performance on {task_name} 
    and provide the training instances to teach the student to become more expert.

    Task Context:
    {task_description}

    I will present you with an instance where the student provided incorrect responses. The classification labels are defined as follows: {{"FAVOR": 0, "NONE": 1, "AGAINST": 2}}.

    Please consider the potential weakness of the student considering the incorrect response and provide a training instances to teach it to be a more expert problem solver. 

    Format your training instances using XML tags.
    The <TrainingSet> should include {training_num} questions, each enclosed in <train i>, where 'i' is the sequential number of the training set. 
    The <text>, <target>(an entity in the text), and <ground_truth>(Truth label) elements should be within each <train i>.

    Ensure that the length of the text varies (the length exceeds 100) and that the diversity of the text is also guaranteed 
    (you can choose materials from different fields or different tones like diverse users on the website).

    The data are as follows:
    {data}
    """

    batch = 1
    training_num = 10

    task_name = "stance detection"

    task_description = "Stance detection aims to identify the authors' attitudes or positions [FAVOR, NONE, AGAINST] towards a specific target such as an entity, a topic."

    # print(prompt.format(training_num=training_num, task_name = task_name, task_description = task_description, data=prepare_data(data,num_data)))
    answer = generate_answer(prompt.format(training_num=training_num, task_name = task_name, task_description = task_description, data=prepare_data(data,num_data)))
    # print(answer)
    return answer
def parse(answer:str):
    pattern = r"```xml(.*?)```"
    match = re.search(pattern, answer, re.DOTALL)
    if match:
        extracted_content = match.group(1).strip()
        print("Extracted Content:")
        print(extracted_content[:20],".....")
    else:
        print("No content found between ```xml and ```")
    try:
        root = ET.fromstring(extracted_content)
    except Exception as e:
        print("error: ", answer[:40])
        return [answer]
    js = []
    for i in root:
        k = {}
        for j in i:
            k[j.tag] = j.text
        js.append(k)
    print("parse successfully: len(parse_json) = ",len(js))
    return js
def repeat_generate(raw_data,index,num:int):
    if num > 2:
        print(f"error : don't generate resolvable answer, index: {index}")
        return [index]
    print("repeat num: ", num)
    answer = get_answer(raw_data,index)
    parse_result = parse(answer)
    if len(parse_result) < 10:
        return repeat_generate(raw_data,index,num+1)
    return parse_result
response_data_copy = copy.deepcopy(response_data)
for index, (response, raw) in enumerate(zip(response_data, raw_data)):
    if len(response) < 10:
        response_data_copy[index] = repeat_generate(raw_data,index,0)

with open("./data/all_error_instances_weakness_generate_train_dataset_2.json","w") as f:
    json.dump(response_data_copy,f,indent=4)

In [ ]:
# with open("./data/all_error_instances_weakness_generate_train_dataset.json","r") as f:
#     response_data = json.load(f)

# for i in response_data:
#     if len(i) < 10:
#         print(i[0])
# with open("./data/all_error_instances_weakness_generate_train_dataset_2.json","w") as f:
#     json.dump(response_data_copy,f,indent=4)
with open("./data/all_error_instances_weakness_generate_train_dataset_2.json","r") as f:
    data = json.load(f)

len(data)
sum = 0
l = []
for i in data:
    if len(i) == 10:
        for j in i:
            # print(j)
            if j["ground_truth"] != '0' and j["ground_truth"] != '1' and j['ground_truth'] != '2' and j['ground_truth'] not in l:
                print(j['ground_truth'])
                l.append(j['ground_truth'])
            # if list(j.keys()) != ['text', 'target', 'ground_truth']:
            #     print(j.keys())

In [14]:
import csv
import json
with open("./data/train/3.1_test1/raw_response_parse_iter3.json","r") as f:
    data = json.load(f)
# data
transfer = {
    '0': "AGAINST",
    '1': "FAVOR",
    '2': "NONE",
    'FAVOR': "FAVOR",
    'NONE': "NONE",
    'AGAINST': "AGAINST"
    
}
write_data = []

for j in data:
    # if len(i) == 10:
        # for j in i:
        try:
            s = []
            s.append(j["text"])
            s.append(j["target"])
            s.append(transfer[j["ground_truth"].strip(" ")])
            s.append(1)
            s.append("invalid")
            write_data.append(s)
        except Exception as e:
            print(j)
len(write_data)
csv_filename = './data/train/3.1_test1/raw_response_parse_iter3.csv'
with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Tweet', 'Target 1', 'Stance 1', 'seen?', 'GT Target'])  # 写入标题行
    writer.writerows(write_data)  # 写入数据行


{'index_response': 28, 'index_stance': 5, 'text': "I'm not sure if I would cancel my online subscription to the Times if they didn't improve their comment section. I need to weigh the pros and cons.", 'target': 'comment section', 'ground_truth': ''}
{'index_response': 28, 'index_stance': 15, 'text': "I'm not sure if the Times is censoring comments or not. I need more information to form an opinion.", 'target': 'comment section', 'ground_truth': ''}
{'index_response': 28, 'index_stance': 25, 'text': "I'm not sure if the Times is unduly influencing public opinion or not. I need more information to form an opinion.", 'target': 'comment section', 'ground_truth': ''}
{'index_response': 45, 'index_stance': 15, 'text': "I'm not sure what to make of public opinion polls. Sometimes they seem accurate, but other times they're way off.", 'target': 'democracy', 'ground_truth': ''}
{'index_response': 45, 'index_stance': 19, 'text': "I'm not sure how much stock to put in public opinion polls. Someti

In [ ]:
# csv_filename = './data/all_error_instances_weakness_generate_train_dataset_2.csv'
# import csv


# # 打开CSV文件
# with open(csv_filename, mode='r', encoding='utf-8') as file:
#     # 创建CSV阅读器
#     csv_reader = csv.reader(file)
    
#     # 遍历CSV文件中的每一行
#     for row in csv_reader:
#         print(row)  # 打印每一行，row是一个列表
import json
path = "/home/ubuntu/wcc/now-task/scripts/results/baseline/checkpoint-318/log_train_parse_1_60_wrong_test_data.json"
with open(path,"r") as f:
    data = json.load(f)
index = []
for i in range(527):
    index.append([0]*3)
    for x in data:
        if x["index"] >= i*60 and x["index"] < (i+1)*60:
            index[i][(x["index"]-i*60)//20] += 1
ans = []
for index_i,i in enumerate(index):
    for index_j,j in enumerate(i):
        if j == 20:
            ans.append(index_i*60+index_j*20)
print(len(ans))
ans
with open("/home/ubuntu/wcc/now-task/data/train/spurious_1_60_parse.json","r") as f:
    parse = json.load(f)
final_data = []
for i in ans:
    final_data += parse[i:i+20]

with open("/home/ubuntu/wcc/now-task/data/train/train_spurious_1_60_parse_100%.json","w") as f:
    json.dump(final_data,f,indent=4)
    
    


In [2]:
a = [1,2]
a[0:0]

[]

In [15]:
import pandas as pd
path1 = "./data/train/3.1_test1/raw_response_parse_iter3.csv"
path2 = "./data/train/3.1_test1/raw_response_parse_combined_iter2.csv"
# 读取两个 CSV 文件
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

# 按行合并，自动对齐列
combined_df = pd.concat([df1, df2], ignore_index=True)

# 保存结果
combined_df.to_csv("./data/train/3.1_test1/raw_response_parse_combined_iter3.csv", index=False)
print(combined_df)


                                                   Tweet  \
0      It's astonishing how some people blame their c...   
1      The concept of personal accountability is cruc...   
2      It's disappointing to see how some people lack...   
3      Personal accountability is essential for build...   
4      It's refreshing to see individuals who take pe...   
...                                                  ...   
57289  The United States of America is the most advan...   
57290  The United States of America is the most advan...   
57291  I'd just like to see the curriculum. Too many ...   
57292  Analogies are a weak form of argument, but the...   
57293  I personally believe that whatever the outcome...   

                      Target 1 Stance 1  seen? GT Target  
0              personal growth    FAVOR      1   invalid  
1             self-improvement    FAVOR      1   invalid  
2             entrepreneurship    FAVOR      1   invalid  
3        relationship building    FAVOR    

In [11]:

import json
import matplotlib.pyplot as plt
path = "/home/ubuntu/wcc/now-task/scripts/results/baseline/checkpoint-318/log_batch_request_wrong_test_data.json"
with open(path,"r") as f:
    data = json.load(f)
    
y = []

index = []
for i in range(527):
    index.append([0]*3)
    for x in data:
        if x["index"] >= i*15 and x["index"] < (i+1)*15:
            index[i][(x["index"]-i*15)//5] += 1
# print(index)
ans = []
for index_i,i in enumerate(index):
    for index_j,j in enumerate(i):
        if j <= 3 and j >=1:
            ans.append(index_i*15+index_j*5)
print(len(ans))

with open("/home/ubuntu/wcc/now-task/data/train/batch_request/raw_response_1_15_parse.json", "r") as f:
    raw_data = json.load(f)

d = []
for i in ans:
    for j in range(5):
        d.append(raw_data[i+j])
print(len(d))
with open("/home/ubuntu/wcc/now-task/data/train/batch_request/raw_response_1_15_parse_20-60%.json","w")  as f:
    json.dump(d,f,indent=4)
    

#     y.append(len(ans))
# x = [_*5 for _ in range(6)]
# print(len(x),len(y))
# print(sum(y))
# print(x)
# print(y)
# # 创建折线图
# plt.plot(x, y, label="Prime Numbers", color="blue", marker="o")
# # 添加标题和标签
# plt.title("前百分比错误率错误数")
# plt.xlabel("X-axis")
# plt.ylabel("Y-axis")
# # 显示图例
# plt.legend()
# # 显示图表
# plt.show()
            


469
2345


In [ ]:
import json
with open("/home/ubuntu/wcc/now-task/data/train/train_spurious_1_60_parse_100%.json","r") as f:
    data = json.load(f)
s = [0]*3
for i in data:
    if i["ground_truth"] == "1":
        s[1] += 1
    if i["ground_truth"] == "0":
        s[0] += 1
    if i["ground_truth"] == "2":
        s[2] += 1
s

In [ ]:
import pandas as pd
data = pd.read_csv("/home/ubuntu/wcc/now-task/data/train/train_spurious_1_60_parse_combain_100%.csv")
s = [0]*3
for _, d in data.iterrows():
    if d["Stance 1"] == "NONE":
        s[1] += 1
    if d["Stance 1"] == "FAVOR":
        s[0] += 1
    if d["Stance 1"] == "AGAINST":
        s[2] += 1
s

In [ ]:
baseline = "/home/ubuntu/wcc/now-task/scripts/results/baseline/checkpoint-318/wrong_test_data.json"
path_50 = "/home/ubuntu/wcc/now-task/scripts/results/spurious_1_60/checkpoint-342/log_testset_wrong_test_data.json" 
path_90 = "/home/ubuntu/wcc/now-task/scripts/results/spurious_1_60_90%/checkpoint-348/log_testset_wrong_test_data.json"
path_100 = "/home/ubuntu/wcc/now-task/scripts/results/spurious_1_60_100%/checkpoint-696/log_testset_wrong_test_data.json"
import json
def get_data(path):
    with open(path,"r") as f:
        data = json.load(f)
    return data
def get_same_data(baseline,compare):
    b = get_data(baseline)
    c = get_data(compare)
    sum = 0
    for i in b:
        for j in c:
            if i["index"] == j["index"]:
                sum += 1
    return sum/len(b)      
print(len(get_data(baseline)))
print(len(get_data(path_50)))
print(len(get_data(path_90)))
print(len(get_data(path_100)))

print("val 错误率 50%",get_same_data(baseline,path_50))
print("val 错误率 90%",get_same_data(baseline,path_90))
print("val 错误率 100%",get_same_data(baseline,path_100))


In [ ]:
raw_path = "./scripts/results/baseline/checkpoint-318/log_dev_wrong_test_data.json" # prompt给的数据
response_error_path = "/home/ubuntu/wcc/now-task/scripts/results/baseline/checkpoint-318/log_train_parse_1_60_wrong_test_data.json" # baseline答错的数据
response_raw_path= "/home/ubuntu/wcc/now-task/data/train/spurious_1_60.json" # 3部分
index = []
import json
with open(raw_path,"r") as f:
    raw_data = json.load(f)
with open(response_raw_path,"r") as f:
    raw_response = json.load(f)
    
with open(response_error_path,"r") as f:
    data = json.load(f)
def extract(data,num):
    text = data.strip("```xml").strip("```").split("\n\n")
    return text[num]      
for i in range(527):
    index.append([0]*3)
    for x in data:
        if x["index"] >= i*60 and x["index"] < (i+1)*60:
            index[i][(x["index"]-i*60)//20] += 1
ans = []
final_data = []
for index_i,i in enumerate(index):
    x = {}
    x["raw_question"] = raw_data[index_i]["input_text"]
    x["raw_target"] = raw_data[index_i]["target"]
    x["raw_true_label"] = raw_data[index_i]["true_label"]
    x["raw_predict_label"] = raw_data[index_i]["predicted_label"]
    x["pattern"] = ""
    for index_j,j in enumerate(i):
        if j >= 20: # 控制百分比
            x["pattern"] += extract(raw_response[index_i],index_j)
            
    if x["pattern"] != "":
        # x["pattern"] = json.load(x["pattern"])
        final_data.append(x)
final_data
with open("spurious_1_60_100%_with_raw_question_and_pattern_data.json","w") as f:
    json.dump(final_data,f, ensure_ascii=False,indent=4)            
            
     







In [4]:
ls = []
current_results_path = 1
test_path = 2
sign = 3
ls.append(
            f"python eval.py "
            f"--model_root_path '{current_results_path}' "
            f"--test_path '{test_path}' "
            f"--output_error_question False "
            f"--save_log True "
            f"--sign '{sign}'"
)
ls[0]

"python eval.py --model_root_path '1' --test_path '2' --output_error_question False --save_log True --sign '3'"

In [3]:
import os

def generate_file_structure(path):
    """递归遍历文件夹，生成文件结构"""
    structure = []
    
    # 遍历文件和文件夹
    for root, dirs, files in os.walk(path):
        # 计算当前目录的深度
        depth = root.replace(path, '').count(os.sep)
        indent = '│   ' * depth  # 每级目录缩进4个字符
        folder_name = os.path.basename(root)
        structure.append(f"{indent}├── {folder_name}/" if folder_name else '')
        
        # 添加文件
        for file in files:
            structure.append(f"{indent}│   └── {file}")  # 文件的缩进方式
    return structure

def write_to_readme(file_structure, output_file='README.md'):
    """将文件结构写入 README.md"""
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("# 项目文件结构\n\n")
        f.write("```plaintext\n")
        f.write("\n".join(file_structure))
        f.write("\n```\n")

# 使用时指定项目路径
project_path = '.'  # 当前路径，也可以修改为你的项目根目录路径
file_structure = generate_file_structure(project_path)
write_to_readme(file_structure)

